# Categorial Variables with Linear and Nonlinear Models

Three types of variables are used, particularly in linear regression.

1. Continuous - ordered and can be subdivided.
2. Categorical - limited and fixed number of values.
3. Ordinal - limited and fixed number of values for which order is important.

Categorical variables are sometimes referred to as *levels* (particularly in R). They are often called [*dummy variables*](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)) in the statistics literature and indicate membership in a category. But see below.

The typical use case for ordinal variables is a survey where `1` represents "strongly disagree" and `5` represents "strongly agree". A later talk will address the use of ordinal variables in modeling.

The actual numbers for categorical variables do not matter.

## Differences in Languages (Packages)

We have to encode variables for use in `scikit-learn` as the package only takes numeric categories. `R` takes "levels" and encodes internally. So you can pass strings as categories to models in `R`, but not in `scikit-learn`.

# Transforming Variables in Pandas

In [2]:
import numpy as np
import pandas as pd

from random import choice

In [3]:
df = pd.DataFrame(np.random.randn(25, 2), columns=['a', 'b'])
df['e'] = [choice(('Chicago', 'Boston', 'New York', "RALEIGH", "Raleigh", "NYC")) for i in range(df.shape[0])]
df.head(6)

,a,b,e
0,-0.185217,0.606496,NYC
1,-0.267025,-0.713918,NYC
2,-0.098341,-0.091022,New York
3,1.078123,1.231071,Raleigh
4,-0.844573,0.830205,NYC
5,-0.358911,-1.686586,Raleigh


`scikit-learn` has the classes [`OneHotEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) and [`LabelEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) in `sklearn.preprocessing`. `OneHotEncoder` encodes columns that have only integers. `LabelEncoder` encodes strings. Note that there are some complications when using these classes so read the documentation and examples.

Here is another way with Pandas. Use it as a preprocessing step before using `scikit-learn` models.

In [4]:
df1 = pd.get_dummies(df, prefix=["e"])
df1.head(6)

,a,b,e_Boston,e_Chicago,e_NYC,e_New York,e_RALEIGH,e_Raleigh
0,-0.185217,0.606496,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.267025,-0.713918,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.098341,-0.091022,0.0,0.0,0.0,1.0,0.0,0.0
3,1.078123,1.231071,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.844573,0.830205,0.0,0.0,1.0,0.0,0.0,0.0
5,-0.358911,-1.686586,0.0,0.0,0.0,0.0,0.0,1.0


Pandas did what we asked, but unfortunately, some of our variables were not encoded properly. We should have looked at the data first. There are upper/lower case problems, an embedded space, and 'NYC' is encoded 'New York' as well.

In [6]:
df.groupby(by="e").count()

,a,b
e,,
Boston,2,2
Chicago,7,7
NYC,9,9
New York,3,3
RALEIGH,2,2
Raleigh,2,2


Clean it up, then use ``get_dummies`` again.

Examine your categorial variables to avoid creating redundant features.

What should you do with NA in the categorial field? Create a new column which encodes whether a variable is missing or not. This is particularly important for nonlinear models, such as Random Forests.

## Categorical Variables and Linear Models

In models such as regression, which are linear in the unknown variables, we can't include a variable for all categories. If we have four dummy variables for quarter, we must only include three of them.

- 1 Quarter = 1 or 0
- 2 Quarter = 1 or 0
- 3 Quarter = 1 or 0
- 4 Quarter = 1 or 0

Encoding zero for quarter 1 - 3 is the same as encoding a 1 for quarter 4 and the rest zero. Remember, linear regression has an *intercept*, which is why we can't use four dummy variables.

The dummy variable that is omitted is the base category against which all others are compared.

For instance

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$ln(\tt{wage}) = \alpha + \beta \tt{college} + \tt{error}$

Here we set up a regression problem with `college` as one or zero. If `college` is one, then the wage goes up (assuming $\beta > 0$). So we can interpret the significance of the `college` variable as measuring a wage premium/discount for attending college.

The `error` term contains

1. Every variable not included in the regression model.
2. Randomness.

Policy makers like such models. They are not necessarily interested in prediction, and estimating models in these contexts is very different from prediction.

We're not going to actually estimate such a model - that is an hour topic in itself.

Questions

1. Why am I taking the natural log of `wage`?
2. What happens if $\beta < 0$?
3. We encoded `college` as zero or one. Why not one and two?

Including all dummy variables in a regression model introduces [*multicollinearity*](https://en.wikipedia.org/wiki/Multicollinearity) and can cause all kinds of problems for predictions. It *may* be ok for an explantory model, if appropriate corrections are made.

Logistic regression has the same problem.

Note you can remove the intercept, but this may introduce complications in your modeling. For instance, in ``R``, the intercept is included whenever you have a formula like ``y ~ x`` for use in ``lm``, so you **must** use ``y ~ x - 1``. It is easy to forget this.

Of course, you should seriously consider scaling or transforming your data when using a linear model. This is especially important when using features such as wages and a categorical variable as above. But scaling or transforming categorical variables is not a good idea.

## Nonlinear Models

When you fit a nonlinear model, such as a Random Forest, you must include all categories. There is no concept of an intercept term.